In [1]:
import pandas as pd 

df=pd.read_csv('Processed.csv')       

print ("DataFrame shape: ", df.shape)
df.head(5)              

DataFrame shape:  (2028, 39)


,Age,Gender,University,Department,Academic_Year,Current_CGPA,waiver_or_scholarship,PSS1,PSS2,PSS3,...,PHQ2,PHQ3,PHQ4,PHQ5,PHQ6,PHQ7,PHQ8,PHQ9,Depression Value,Depression Label
0,18-22,Female,"Independent University, Bangladesh (IUB)",Engineering - CS / CSE / CSC / Similar to CS,Second Year or Equivalent,2.50 - 2.99,No,3,4,3,...,2,3,2,2,2,2,3,2,20,Severe Depression
1,18-22,Male,"Independent University, Bangladesh (IUB)",Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,3,3,4,...,2,2,2,2,2,2,2,2,19,Moderately Severe Depression
2,18-22,Male,American International University Bangladesh (...,Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,0,0,0,...,0,0,0,0,0,0,0,0,0,No Depression
3,18-22,Male,American International University Bangladesh (...,Engineering - CS / CSE / CSC / Similar to CS,Third Year or Equivalent,3.00 - 3.39,No,3,1,2,...,1,2,1,2,1,2,2,1,14,Moderate Depression
4,18-22,Male,North South University (NSU),Engineering - CS / CSE / CSC / Similar to CS,Second Year or Equivalent,2.50 - 2.99,No,4,4,4,...,3,3,3,1,3,0,3,3,20,Severe Depression


In [2]:
# count duplicates and nans
print ("Duplicates: ", df.duplicated().sum())   
if sum(df.isnull().sum()) == 0:
    print ("No nans")
else:
    print ("Nans: ", df.isnull().sum())

# delete duplicates
df.drop_duplicates(inplace=True)

Duplicates:  6
No nans


In [3]:
#saved cleaned data
df.to_csv('Cleaned.csv', index=False)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Remove remaining non-numeric label columns to avoid errors
data_cleaned = df.drop(columns=["Stress Label", "Anxiety Label"])

# Copy original dataframe
df_transformed = df.copy()

# Convert Age ranges to numeric midpoints
age_map = {
    "Below 18": 17,
    "18-22": 20,
    "23-26": 24.5,
    "27-30": 28.5,
    "Above 30": 32
}
df_transformed["Age"] = df_transformed["Age"].map(age_map)

# Convert CGPA ranges to midpoints
cgpa_map = {
    "Below 2.50": 2.25,
    "2.50 - 2.99": 2.75,
    "3.00 - 3.39": 3.20,
    "3.40 - 3.79": 3.60,
    "3.80 - 4.00": 3.90,
    "Other": np.nan
}
df_transformed["Current_CGPA"] = df_transformed["Current_CGPA"].map(cgpa_map)

# Convert Map Academic Year to numeric year
academic_year_map = {
    "First Year or Equivalent": 1,
    "Second Year or Equivalent": 2,
    "Third Year or Equivalent": 3,
    "Fourth Year or Equivalent": 4,
    "Other": 0
}
df_transformed["Academic_Year"] = df_transformed["Academic_Year"].map(academic_year_map)

# Binary encode waiver_or_scholarship
df_transformed["waiver_or_scholarship"] = df_transformed["waiver_or_scholarship"].map({"Yes": 1, "No": 0})

# One-hot encode Gender, University, Department
df_transformed = pd.get_dummies(df_transformed, columns=["Gender", "University", "Department"], drop_first=True)

# Display the transformed dataset structure
print(df_transformed.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2022 entries, 0 to 2027
Data columns (total 63 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Age                                                                    2022 non-null   float64
 1   Academic_Year                                                          2022 non-null   int64  
 2   Current_CGPA                                                           1851 non-null   float64
 3   waiver_or_scholarship                                                  2022 non-null   int64  
 4   PSS1                                                                   2022 non-null   int64  
 5   PSS2                                                                   2022 non-null   int64  
 6   PSS3                                                                   2022 non-null   int64 

In [6]:
missing = df_transformed.isnull().sum() / df_transformed.shape[0] * 100
missing[missing > 0]

Current_CGPA    8.456973
dtype: float64

In [7]:
df_transformed.select_dtypes(exclude="object").dropna().corr().Current_CGPA.sort_values(ascending=False)


Current_CGPA                                                             1.000000
University_Islamic University of Technology (IUT)                        0.182567
PSS8                                                                     0.182476
waiver_or_scholarship                                                    0.158758
PSS5                                                                     0.122492
PSS6                                                                     0.102125
Department_Engineering - EEE/ ECE / Similar to EEE                       0.092149
PSS7                                                                     0.069163
University_Rajshahi University (RU)                                      0.054589
University_Dhaka University (DU)                                         0.041322
PSS4                                                                     0.040819
PHQ4                                                                     0.039706
Gender_Prefer no

In [22]:
# Separate features and target again
X = data_cleaned.drop(columns=["Depression Label"])
y = data_cleaned["Depression Label"]

# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (stratified to maintain label distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Output dimensions and class names
le = LabelEncoder()
X_train.shape, X_test.shape, y_train.shape, y_test.shape, le.classes_


ValueError: could not convert string to float: 'Female'